In [197]:
import requests
import re
import pandas as pd

In [198]:
import os
from dotenv import load_dotenv

load_dotenv()

True

In [199]:
api_key = os.getenv("lechat_api_key")

In [200]:
# api_key = "TqwpZsnTmT1WQG5A6T3fzDomj4PwZ0hn" # Replace with your actual API key

In [201]:
def ask_le_chat(api_key, question, temperature=0.7):
    api_endpoint = "https://api.mistral.ai/v1/chat/completions"

    headers = {
        "Authorization": f"Bearer {api_key}",
        "Content-Type": "application/json"
    }

    data = {
        "model": "mistral-tiny", # or another engine if available
        "messages": [{"role": "user", "content": question}],
        "temperature": temperature
    }

    try:
        response = requests.post(api_endpoint, headers=headers, json=data)
        response.raise_for_status()  # Raises an HTTPError for bad responses

        # Assuming the response contains the answer in a JSON format
        answer = response.json()
        return answer.get('choices', [{}])[0].get('message', {}).get('content', 'No answer found')

    except requests.exceptions.RequestException as e:
        return f"An error occurred: {e}"


In [ ]:
# 1. 확률 답변 요청

question = """
2025년 5월 1일부터 오늘까지의 한글/영문/중문문 기사 중 니켈 가격 변동을을 조사한 후,
답변은 니켈 가격이 상승할 확률과 니켈 가격이 하락할 확률 숫자만 보여주세요.
예시) 니켈 가격이 상승할 확률: 70%, 하락 확률: 30%
"""

In [203]:
answers = []
for i in range(10):
    answer = ask_le_chat(api_key, question, temperature=0.9)
    answers.append(answer)
    print(answer)


정보를 조사하여 가져온 결과입니다.

한국 원화의 통화가치가 상승할 확률: 60%, 하락 확률: 40%

이 결과는 오늘 기준으로 예측되며, 예측 결과는 장기적인 향후 예측이 아닌 오늘부터의 단기적인 예측입니다. 이 결과는 개별적인 예측이며, 금융시장 상황의 변화로 인하여 예측 결과는 변수가 될 수 있습니다. 이 결과는 투자 관련 정보이므로 투자 판단에 절대 iewm-multiplex-101 자료로 사용할 것을 推하지 않습니다. 항상 자세한 정보와 전문적인 조언을 받으시기 바랍니다.
이 질문에 대한 정보를 제공하기 위해 2025년 5월 1일부터 현재까지의 기사를 조사하여, 한국 원화의 통화가치가 상승할 확률과 하락할 확률을 추정했습니다.

한국 원화의 통화가치가 상승할 확률: 60%,
한국 원화의 통화가치가 하락할 확률: 40%

이 정보는 기사의 내용을 바탕으로 생성된 추정치입니다. 이 정보는 정확한 분석이 아닙니다. 자금시장 분석을 위해서는 전문가와 상담하는 것이 바람직합니다.
이 문제에 대한 답변은 현재 2022년 12월 30일 기준으로 데이터를 분석한 결과를 바탕으로 하여 맥락상 맞추어지는 답변을  drasticly fluctuate 없이 기수(기준)를 두고 예측해 보았습니다.

한국 원화의 통화가치가 상승할 확률: 60%, 하락 확률: 40% (이 통계치는 2022년 12월 30일 기준으로 분석된 것으로, 이후 변동이 있을 수 있음을  bilking 맞추지 않으셨습니다.)

통화 가치 상승은 대두 환율이 오랫동안 상승하면서 도달한 지금의 되삵 와전 가격 및 발전 과다로 인해 연귀 통화 수익이 증가할 확률이 높습니다. 하지만 전자 상거래의 세계장화와 인플레이션 시장의 여롱증 등 다음 몇 년 동안에도 불확실성이 존재할 것으로 예상됩니다.

한국 원화의 통화가치가 하락할 확률의 원인으로는 외주 투자의 감소, 금융 시장의 변동, 국내 경제 비정상 증식 등이 가장 큰 원인으로 간주되며, 이러한 맹 moins인화와 같은 사업에서 원화 가격이 낮아짐으로써 통화 가

In [204]:
def extract_first_two_percent_values(text):
    """
    주어진 텍스트에서 앞에 등장하는 두 개의 '숫자%' 값을 찾아 숫자(int)로 반환합니다.
    
    Parameters:
        text (str): 분석할 텍스트
    
    Returns:
        tuple: (첫 번째 정수, 두 번째 정수), 두 개를 못 찾으면 None
    """
    # 정규표현식으로 '숫자%' 형태 추출
    matches = re.findall(r'(\d{2})%', text)
    
    if len(matches) >= 2:
        # 앞에서 두 개만 정수로 변환해서 반환
        return float(matches[0]), float(matches[1])
    else:
        return None

In [205]:
answers_float = []
for text in answers:
    if text is not None:
        temp = extract_first_two_percent_values(text)
        if temp is not None and temp[0] + temp[1] == 100:
            answers_float.append(temp)

In [206]:
answers_float

up, down = 0, 0
for u, d in answers_float:
    up += u
    down += d
up = up / len(answers_float)
down = down / len(answers_float)

answers_float.append((round(up, 1), round(down, 1)))
answers_float

[(60.0, 40.0),
 (60.0, 40.0),
 (60.0, 40.0),
 (65.0, 35.0),
 (75.0, 25.0),
 (90.0, 10.0),
 (60.0, 40.0),
 (60.0, 40.0),
 (60.0, 40.0),
 (65.6, 34.4)]

In [207]:
pd.DataFrame(answers_float, columns=['상승 확률', '하락 확률'], 
             index=[str(i)+"회" for i in range(1, len(answers_float))] + ["평균"]).to_csv("환율_확률_결과.csv", encoding='utf-8')

In [208]:

question = """
2025년 5월 1일부터 오늘까지의 한국어/영어/중국어 기사 중 원/달러 환율과 달러화 내용을을 조사한 후,
최근 기사에서 제시하는 원화의 통화 가치가 상승할 요인 4가지를 (요인 | 설명) [표] 형식 보여주세요.
"""

for i in range(5):
    answer = ask_le_chat(api_key, question, temperature=0.2)
    if "| --- | --- |" in answer:
        break

print(answer)

다음은 2021년 1월 1일부터 2022년 12월 31일까지 발행된 한국어, 영어, 중국어 기사 중 원/달러 환율과 달러화 관련 내용을 조사한 결과, 최근 기사에서 제시하는 원화의 통화 가치가 상승할 요인 4가지를 (요인 | 설명) 형식으로 보여드리겠습니다.

| 요인 | 설명 |
| --- | --- |
| 1. 경제 성장 | 국내 경제 성장이 빠르게 진행되면, 외국인 투자자들이 한국 기업에 투자하기 쉬워지고, 원화 공급이 증가하게 되어 원화의 가치가 상승할 수 있습니다. |
| 2. 통화 정책 | 중앙은행이 통화 정책을 통해 원화 공급을 제한하거나 원화 가치를 지원하는 방법을 사용할 수 있습니다. 예를 들어, 중앙은행이 통화 정책을 통해 원화 공급을 제한하면, 원화 공급이 줄어들고 원화의 가치가 상승할 수 있습니다. |
| 3. 외국 통화 가치 하락 | 외국 통화의 가치가 하락하면, 해당 국가의 상품 가격이 높아지고, 해당 국가의 기업에 대한 투자가 쉬워지게 됩니다. 이로 인해 원화 공급이 증가하고, 원화의 가치가 상승할 수 있습니다. |
| 4. 국제 경제적 조건 | 국제 경제적 조건이 좋아지면, 국내 경제 성장이 빠르게 진행되고, 외국인 투자가 증가하게 됩니다. 이로 인해 원화 공급이 증가하고, 원화의 가치가 상승할 수 있습니다. 예를 들어, 국제 경제적 조건이 좋아지면, 국내 기업의 수출이 증가하고, 외국인 투자자들이 한국 기업에 투자하기 쉬워지게 됩니다. |


In [209]:
contents = []
for i, text in enumerate(answer.split("\n")):
    if i == 0:
        continue
    if "| --- | --- |" in text:
        continue
    if "|" in text:
        temp = text.split("|")[1:3]
        contents.append(temp)

In [210]:
pd.DataFrame(contents, columns=['요인', '설명']).to_csv("원화_강세_요인.csv", encoding='utf-8', index=False)

In [211]:
question = """
2025년 5월 1일부터 오늘까지의의 한국어/영어/중국어 기사 중 원/달러 환율과 달러화 내용을을 조사한 후,
최근 기사에서 제시하는 원화의 통화 가치가 하락할 요인 4가지를 (요인 | 설명) [표] 형식 보여주세요.
"""

for i in range(5):
    answer = ask_le_chat(api_key, question, temperature=0.2)
    if "| --- | --- |" in answer:
        break

print(answer)

| 요인 | 설명 |
| --- | --- |
| 1. 경제 불안정성 | 국내 경제 성장 속도의 저하, 외국 경제 불안정성 등이 원화의 가치를 하락시키는 요인입니다. 경제 불안정성은 투자자들의 원화 투자 희망을 떨어뜨리고, 외국 투자자들이 원화를 팔아 달러를 구입하게 되어 원화의 가치가 하락합니다. |
| 2. 통화 정책 | 중앙은행의 통화 정책이 원화 가치를 하락시키는 요인입니다. 중앙은행이 통화 혹은 이자율을 높이거나 낮추는 등의 조치로 원화의 가치를 하락시킬 수 있습니다. 이는 투자자들이 원화를 팔아 달러를 구입하게 되어 원화의 가치가 하락합니다. |
| 3. 국제 경제 조건 | 국제 경제 조건이 원화의 가치를 하락시키는 요인입니다. 예를 들어, 외국 경제 성장 속도의 저하, 외국 통화 가치의 하락 등이 원화의 가치를 하락시키는 요인입니다. 이는 투자자들이 원화를 팔아 달러를 구입하게 되어 원화의 가치가 하락합니다. |
| 4. 정치적 위험 | 정치적 위험이 원화의 가치를 하락시키는 요인입니다. 예를 들어, 국내 정치적 불안정성, 국제 정치적 불안정성 등이 원화의 가치를 하락시키는 요인입니다. 이는 투자자들이 원화를 팔아 달러를 구입하게 되어 원화의 가치가 하락합니다. |


In [212]:
contents = []
for i, text in enumerate(answer.split("\n")):
    if i == 0:
        continue
    if "| --- | --- |" in text:
        continue
    if "|" in text:
        temp = text.split("|")[1:3]
        contents.append(temp)

In [213]:
pd.DataFrame(contents, columns=['요인', '설명']).to_csv("원화_약세_요인.csv", encoding='utf-8', index=False)